In [5]:
from matplotlib import cm
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np

# Problem 1
Taken from Rao (2009). Engineering Optimization: Theory and Practice, 4th Ed. USAL John Wiley & Sons, Ltd.

The layout of a processing plant consisting of a pump (P), a water tank (T), a compressor (C), and a fan (F), is shown. The locations of the various units in terms of their $(x, y)$ coordinates are also indicated. It was decided to add a new unit, a heat exchanger (H), to the plant. To avoid congestion, it was decided to locate H within a rectangular area defined by $15\leq x \leq 15$ and $-10\leq y \leq 10$. Find the location of H that minimizes the sum of its Euclidean distances from the existing units, P, T, C, and F.

In [6]:
model = ConcreteModel()
model.x = Var(initialize=0.0, bounds=(-15, 15))
model.y = Var(initialize=0.0, bounds=(-10, 10))

def sum_dist(m):
    return sqrt((m.x+25)**2 + (m.y+35)**2) + \
           sqrt((m.x+30)**2 + (m.y-20)**2) + \
           sqrt((m.x-40)**2 + (m.y-30)**2) + \
           sqrt((m.x-20)**2 + (m.y+15)**2)


model.obj = Objective(rule=sum_dist, sense=minimize)
res = SolverFactory('cyipopt').solve(model, tee=True)
model.pprint()

Please recompile / update your pynumero_ASL library.

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables w

# Problem 2
Taken from EHL Problem 8.41

For the purposes of planning you are asked to determine the optimal heat exchanger areas for the sequence of 3 exchangers as shown. 

|Exchanger| U   | Area | Duty|
|---------|-----|------|-----|
| 1       | 120 | $A_1$|$Q_1$|
| 2       |  80 | $A_2$|$Q_2$|
| 3       |  40 | $A_3$|$Q_3$|

Given that $mC_p = 10^5 Btu/h/degF$, find the temperatures $T_1$, $T_2$, and $T_3$ such that $\Sigma A_i$ is a minimum. Assume that the following design equation is valid: $Q=UA\frac{\Delta T_1 + \Delta T_2}{2}$ for ends 1 and 2 in the heat exchanger.

In [42]:
model = ConcreteModel()
model.T1 = Var(initialize=200.0, domain=NonNegativeReals, bounds=(100, 300))
model.T2 = Var(initialize=400.0, domain=NonNegativeReals, bounds=(None, 500))
model.T3 = Var(initialize=500.0, domain=NonNegativeReals, bounds=(None, 600))
model.T4 = Var(initialize=300.0, domain=NonNegativeReals, bounds=(None, 400))
model.T5 = Var(initialize=200.0, domain=NonNegativeReals, bounds=(100, 300))
model.A1 = Var(initialize=10.0, domain=NonNegativeReals)
model.A2 = Var(initialize=10.0, domain=NonNegativeReals)
model.A3 = Var(initialize=10.0, domain=NonNegativeReals)

def sum_areas(m):
    return m.A1 + m.A2 + m.A3

model.obj = Objective(rule=sum_areas, sense=minimize)

def design(m, Th_in, Th_out, Tc_in, Tc_out, U, A):
    #return Th_out-Tc_in == (Th_in-Tc_out)*exp(U*A*((Th_out-Tc_in)-(Th_in-Tc_out))/1e5/(Tc_out-Tc_in))
    #return 1e5*(Tc_out-Tc_in) == U*A*((Th_out-Tc_in)-(Th_in-Tc_out))/log((Th_out-Tc_in)/(Th_in-Tc_out))
    return 1e5*(Tc_out-Tc_in) == U*A*((Th_out-Tc_in)+(Th_in-Tc_out))/2

def balance(m, Th_in, Th_out, Tc_in, Tc_out):
    return Th_out-Th_in == Tc_in-Tc_out

#def LMTD(m, Th_in, Th_out, Tc_in, Tc_out):
#    return Th_out-Tc_in >= Th_in-Tc_out

model.hex1 = Constraint(rule=lambda m: design(m, 300, m.T5, 100, m.T1, 120, m.A1))
model.hex2 = Constraint(rule=lambda m: design(m, 400, m.T4, m.T1, m.T2, 80, m.A2))
model.hex3 = Constraint(rule=lambda m: design(m, 600, m.T3, m.T2, 500, 40, m.A3))

model.bal1 = Constraint(rule=lambda m: balance(m, 300, m.T5, 100, m.T1))
model.bal2 = Constraint(rule=lambda m: balance(m, 400, m.T4, m.T1, m.T2))
model.bal3 = Constraint(rule=lambda m: balance(m, 600, m.T3, m.T2, 500))

#model.LMTD1 = Constraint(rule=lambda m: LMTD(m, 300, m.T5, 100, m.T1))
#model.LMTD2 = Constraint(rule=lambda m: LMTD(m, 400, m.T4, m.T1, m.T2))
#model.LMTD3 = Constraint(rule=lambda m: LMTD(m, 600, m.T3, m.T2, 500))

model.Tcons1 = Constraint(rule=model.T1 <= model.T2)
model.Tcons2 = Constraint(rule=model.T1 <= model.T4)
model.Tcons3 = Constraint(rule=model.T2 <= model.T3)

res = SolverFactory('cyipopt').solve(model, tee=True)
model.pprint()

This is Ipopt version 3.13.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       17
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        8
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  